In [2]:
from tkinter import *
from os.path import exists
import pandas as pd
from odbm.odbm_main import ModelBuilder, ModelHandler
from odbm.utils import extractParams, fmt


To add:
- Events
    - Similar to constant species, have user type in event @ trigger: (A = 3 @ time > 10)
- Sensitivity Analysis
    - user types in species and range of concentrations: A = 1:100, B = .1:1000
- Visualization
    - input species and time (two separate entries)


In [3]:
# Checks if model path given by user exists and builds model.
def loadModel(model_path,root):
    global myModel #create global variable so it is accessible in main
    if exists(model_path) and model_path.endswith(('.xls','.xlsx')):
        model_species = pd.read_excel(model_path, sheet_name = 'Species & Base Mechanisms', engine = 'openpyxl').dropna('index','all')
        model_rxns = pd.read_excel(model_path, sheet_name = 'Reaction', engine = 'openpyxl').dropna('index','all')
        myModel = ModelBuilder(model_species, model_rxns)
        print('Model successfully built')
        modelWindow(myModel, model_path, root)

    else:
        if exists(model_path):
            raise TypeError('Please pass in an excel compatible workbook with sheets named "Species & Base Mechanisms" and "Reaction"')
        else:
            raise FileExistsError('Error: No file exists with given name.')

In [4]:
# 2nd Window
def modelWindow(myModel, model_path, root):

    frame, root = newWindow(root)

    welcome = Label(frame, text = "Model successfully loaded! ")
    welcome.config(font = ('Arial', 12))
    welcome.pack()
    
    cancel_button = Button(frame, text="Cancel", fg="blue",command = lambda: cancelButton(root)) 
    save_button = Button(frame, text = 'Save Model', fg = 'blue', command = lambda: saveModelTxt(myModel,model_path,frame))
    simulate_button = Button(frame, text = 'Simulate Model', fg = 'blue', command = lambda: runSimulation(myModel, model_path, root)) 
    edit_button = Button(frame, text = 'Edit Model', fg = 'blue', command = lambda: EditWindow(model_path, root)) 

    cancel_button.pack(side = LEFT)
    save_button.pack(side = LEFT)
    simulate_button.pack(side = LEFT)
    edit_button.pack(side = LEFT)


In [5]:
def cancelButton(root):
    root.destroy()
    __init__()

def newWindow(root):
    root.destroy()
    root = Tk()
    frame = Frame(root)
    frame.pack()
    return frame, root

def saveModelTxt(myModel,model_path,frame):
    modelfile = model_path[:-4] + 'txt'
    myModel.saveModel(modelfile)
    save_msg = Label(frame, text = 'Model successfully saved.')
    save_msg.pack(side = BOTTOM)
    return modelfile


def runSimulation(myModel, model_path, root):

    # Close previous window and open new
    frame, root = newWindow(root)

    # Set up ModelHandler instance and selections for simulation
    modelfile = saveModelTxt(myModel,model_path, frame)
    with open(modelfile,'r') as m:
        MH = ModelHandler(m.read())
    selections = list(map(fmt, myModel.species['Label'])) + list(myModel.rxns['Label']) + ['time']


    #Add entries, buttons, labels
    start = Entry(frame, width = 10)
    end = Entry(frame, width = 10)
    points = Entry(frame, width = 10)
    titles = Label(frame, text = "Start: \t End: \t Points: \n")
    run_button = Button(frame, text = 'Run Simulation',  fg = 'blue', 
    command = lambda : [MH.setSimParams(int(start.get()),int(end.get()),int(points.get()), selections=selections), 
                        simHelper(MH, model_path, root)])
    back = Button(frame, text = 'Back', fg = 'Blue', command = lambda: modelWindow(myModel, model_path, root ))
    
    titles.pack(side = LEFT)
    start.pack(side = LEFT)
    end.pack(side = LEFT)
    points.pack(side = LEFT)
    run_button.pack(side = RIGHT)
    back.pack(side = RIGHT)

def simHelper(MH, model_path, root):
    # Close previous window and open new
    frame, root = newWindow(root)


    results = MH.rr.simulate(MH.SimParams['start'],MH.SimParams['end'],MH.SimParams['points'],MH.SimParams['selections'])
    feedback = Label(frame, text = "Simulation successfully run")
    feedback.config(font = ('Arial', 10))
    feedback.pack()

    #Add entries, buttons, labels
    back = Button(frame, text = 'Back',  fg = 'blue', command = lambda: runSimulation(myModel,model_path,root))
    save = Button(frame,text = "Save Simulation Results", fg = 'blue', command = lambda: saveSim(results, model_path,frame)) 
    visualize = Button(frame, text = 'Visualize Simulation Results', fg = 'blue') #use vis.py, let user input reactions and/or species

    back.pack()
    save.pack()
    visualize.pack()

def saveSim(results, model_path,frame):
    sim_df = pd.DataFrame(results, columns = results.colnames).round(2)
    results_path = model_path[:-4] + '_RESULTS.xlsx'
    sim_df.to_excel(results_path, sheet_name = 'Results')
    # bug here right now but can't figure it out yet

    feedback = Label(frame, text = "Results successfully saved to excel file")
    feedback.config(font = ('Arial', 10))
    feedback.pack()
    
    


    


In [6]:
def EditWindow(model_path,root):
    
    frame, root = newWindow(root)

    constant = Button(frame, text = 'Add Constant Species', fg = 'blue', command = lambda: setConstantHelper(model_path, root))
    event = Button(frame,text = 'Add Event', fg = 'blue')
    sensitivity = Button(frame, text = 'Sensitivity Analysis', fg = 'blue')
    back = Button(frame, text = 'Back',  fg = 'blue', command = lambda: modelWindow(myModel,model_path,root))


    constant.pack()
    event.pack()
    sensitivity.pack()
    back.pack()


def setConstant(model_path,frame,entry):
    param_dict = {}
    params = entry.split(',')
    for p in params:
        sp,val = p.split(':')
        param_dict[sp] = val

    modelfile = model_path[:-4] + 'txt'
    with open(modelfile,'r') as m:
        MH = ModelHandler(m.read())
    MH.setConstantParams(param_dict)
    
    feedback = Label(frame, text = "Values successfully set")
    feedback.config(font = ('Arial', 10))
    feedback.pack()
    


def setConstantHelper(model_path,root):

    frame, root = newWindow(root)

    prompt = Label(frame, text = "Insert list of species and concentrations in format species:concentration separated by commas")
    prompt.config(font = ('Arial', 12))
    entry = Entry(frame, width = 40)
    set = Button(frame, text = 'Set', fg = 'blue', command = lambda : setConstant(model_path,frame,entry.get()))
    back = Button(frame, text = 'Back',  fg = 'blue', command = lambda: EditWindow(model_path,root))

    prompt.pack()
    entry.pack()
    set.pack()
    back.pack()
    
    

In [7]:
# Main Window
def __init__():
    root = Tk()
    frame = Frame(root)
    frame.pack()

    welcome = Label(frame, text = "Welcome to ODBM \n Please insert file path for model to load:")
    welcome.config(font = ('Arial', 12))
    entry = Entry(frame, width = 40)
    loadbutton = Button(frame, text="Load Model", fg="blue", command = lambda:loadModel(entry.get(),root))

    welcome.pack()
    loadbutton.pack(side = BOTTOM)
    entry.pack()
    root.mainloop()

__init__()

Model successfully built


In [8]:
C:\Users\rycar\Documents\GitHub\ODBM\Examples\model_definition.xlsx

SyntaxError: unexpected character after line continuation character (<ipython-input-8-cf614fb1cf2e>, line 1)